<a href="https://colab.research.google.com/github/jihun212/HAICON2021/blob/main/%EC%B5%9C%EC%A7%80%ED%9B%882.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 최지훈 작성
# 이경연 작성

#최지훈 작성

# 최종 모델 

## 1. Model 구성 
1. 1D CNN(1D Convolutional Neural Networks)
2. MSE Loss + AdamW 

## 2. 최종 성능최적화 방법
1. Model의 ouput에 lowpass filter를 적용해 noise를 제거 (데이터후처리)
2. shingle size와 stride 튜닝                           (데이터전처리)
3. epoch 튜닝                                           (모델 hyper-parameter 조정)
4. Threshold 조정                                       (모델 hyper-parameter 조정)
5. model selection by grid-search

## 3. 모델 선정 기준
1. 일반화 성능이 높은 모델 (Test data' TaPR과 Public score의 차이가 적은 모델)
2. public score가 높은 모델 

## 4. 최종 모델 성능
- Test data TaPR : F1: 0.975 (TaP: 0.997, TaR: 0.953)
- Publice score : 0.97414
- Private score : 0.93414
- 최종 모델 경로 : 첨부 파일의 ./final_submission/final_model_shingle40_epoch70.pt

## 5. 시도한 방법론 (최종 사용X)

- 모델 관점 
    1. LSTM AutoEncoder 모델
    2. RCF(random cut forest) 모델
   
   
- 데이터 전처리관점
    1. feature selection
    2. data augmentation
        - jittering 
        - reverse
        - lstm autoencoder

## 6. Path 정보

- Data Path
    1. traing data : ./hai-master/training/
    2. validation data : ./hai-master/validation/
    3. test data : ./hai-master/testing/
    
    
- Model Path
    1. model 저장 경로 : ./hai-master/{model_name}.pt
    2. 최종제출 model : ./hai-master/final_model_shingle40_epoch70.pt
    
    
- Submission Path
    1. submission 저장 경로 : ./hai-master/
    2. sample path : ./hai-master/sample_submission.csv
    3. 최종제출 submisstion : ./hai-master/final_submission.csv
    
    
- TaPR Lib path
    1. whl file : ./eTaPR-1.12-py3-none-any.whl
    
## 7. Library 버전
- toolz==0.10.0
- torch==1.4.0
- torchvision==0.5.0
- torchviz==0.0.1
- tornado==6.0.4
- numpy==1.18.5
- notebook==6.0.3
- pyparsing==2.4.7
- python 3
- matplotlib 
- pandas
- dateutil
- datetime

## 8. 작업환경
- AWS sagemaker
- instance type : ml.p2.8xlarge

## Import Library

TaPR을 install하고 필요한 library들을 import 합니다.

In [ ]:
import os, sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from datetime import timedelta
import dateutil
from tqdm import tqdm
import easydict
from TaPR_pkg import etapr

## 데이터 전처리

제공된 csv 데이터를 읽어 오는 작업을 진행합니다.

In [ ]:
def dataframe_from_csv(target):
    return pd.read_csv(target).rename(columns=lambda x: x.strip())

def dataframe_from_csvs(targets):
    return pd.concat([dataframe_from_csv(x) for x in targets])

def normalize(df):
    ndf = df.copy()
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]:
            ndf[c] = df[c] - TAG_MIN[c]
        else:
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c])
    return ndf